In [140]:
import numpy as np
import matplotlib.pyplot as plt

class Neural_Network:

    ##NOTES
    ## - the default activation function is a sigmoid.

    ##TO-DO LIST
    ## - add a function to change the activation function

    def __init__(self, structure, thetas = None):

        self.structure = structure;
        self.thetas = thetas;
        
        #create thetas if they do not yet exist
        if(not self.thetas==None):
            self.thetas = thetas;

        else:
            self.thetas = self.genRandThetas();

        self.activationFunction = self.sigmoid;
        self.alpha = 0.5;

    def forwardProp(self, inp):

        result = inp.T;
        result = self.addBias(result);

        values = [];

        for i in range(1, len(self.structure)):
            ##calculate linear combination defined by theta coefficients and apply activation function + add bias
            values.append(result);
            result = self.addBias(self.activationFunction(self.thetas[i-1].dot(result)));


        values.append(result);

        ##return output column minus the bias that was added on the last run of the previous loop
        return values;

    def backProp(self, inp, target): ##currently the inputs are arrays, might change later

        values = self.forwardProp(inp); ##calculate output
        y = target.T;

        ##first layer of thetas
        
        x = values[-2]; ##previous layer
        xm = self.removeBias(values[-1]); ##current layer

        gammas = [(xm-y).T];

        for i in range(0, self.structure[-1]):

            ##i-1 appears where the data structures don't have a bias element to ignore

            theta_i = np.array([self.thetas[-1][i]]);

            self.thetas[-1][i] -= self.alpha * gammas[0][0][i] * x[:,0] * self.dsigmoid(theta_i.dot(x)[0][0])



        ##all of the other layers

        for k in range(len(self.thetas)-2, -1, -1):

            ##calculate the latest gamma
            ##this is the vector of the dsigmoid(...) that is direct-producted into the theta matrix
            pvec = [];
            for r in range(0, len(self.thetas[k+1])):
                pvec.append(self.thetas[k+1][r].dot(values[k+1])[0]);


            pvec = self.dsigmoid(np.array([pvec])).T;

            ##calculate it and add to gammas
            newGamma = gammas[0].dot( (self.thetas[k+1][:,1:] * pvec) );
            gammas = [newGamma] + gammas;

            ##updates thetas
            for i in range(0, len(self.thetas[k])):
                self.thetas[k][i] -= self.alpha * (
                        values[k] * values[k+1][i][0] * (1-values[k+1][i][0]) *
                        gammas[0][0][i])


    def evaluate(self, inp):

        result = inp.T;
        result = self.addBias(result);

        for i in range(1, len(self.structure)):
            ##calculate linear combination defined by theta coefficients and apply activation function + add bias
            result = self.addBias(self.activationFunction(self.thetas[i-1].dot(result)));

        ##return output column minus the bias that was added on the last run of the previous loop
        return self.removeBias(result);

    def genRandThetas(self):

        ##goal: array of matrices of the theta_i,j, each entry of the array corresponds to a matrix which corresponds to the theta_i,j of a layer,
        ##with theta_i,j being the jth theta of the ith node of the given layer

        ##currently thetas are chosen randomly as value between 0 to 1 although this is obviously arbitrary

        thetas = [];

        for i in range(1, len(self.structure)): ##adding a layer of thetas

            layer = [];

            ##for each node in the current layer we add its theta array
            for j in range(0, self.structure[i]):
                currentNodeThetas = [];

                ##for each node in the previous layer we add a corresponding random theta value
                for k in range(0, self.structure[i-1]+1): ##the +1 accounts for a bias node
                    currentNodeThetas.append(np.random.uniform(-1,1));
                
                layer.append(currentNodeThetas);

            thetas.append(np.array(layer));

        return thetas;

    def addBias(self, M):
        return np.concatenate( (np.array([[1]]) , M) );

    def removeBias(self, M):
        return M[1:];

    def sigmoid(self, z):
        return 1/(1+np.exp(-z));

    def dsigmoid(self, z):
        w = self.sigmoid(z);
        return w*(1-w);